# Scopus

Para recuperar los datos de Scopus se va a utilizar la libreria de [pybliometrics](https://pybliometrics.readthedocs.io/en/stable/index.html). Para eso es necesario utilizar una credencial que será almacenada en un archivo de configuración. Más información en [Configuration](https://pybliometrics.readthedocs.io/en/stable/configuration.html)

In [2]:
import os
from pathlib import Path
from os import path

main_path = Path.cwd().parent
os.environ['PYB_CONFIG_FILE'] = "{main_path}/config/scopus_config.ini".format(main_path = main_path)

import pybliometrics

if (not path.exists(os.environ['PYB_CONFIG_FILE'])):
    pybliometrics.scopus.utils.create_config()

Para recuperar datos sobre instituciones de Scopus, se utiliza la [Affiliation Retrieval API
](https://dev.elsevier.com/documentation/AffiliationRetrievalAPI.wadl) y una instancia de la clase [AffiliationRetrieval](https://pybliometrics.readthedocs.io/en/stable/classes/AffiliationRetrieval.html)

In [3]:
import pandas as pd
from pybliometrics.scopus import AffiliationRetrieval

Se recueperan datos de la Universidad Nacional de La Plata a través del identificador **'60032057'** y lo agrego a una lista llamada *affiliations*

In [11]:
affilliations = []

aff = AffiliationRetrieval(60032057)
print(aff)

affilliations.append(aff)

Universidad Nacional de La Plata in La Plata in Argentina,
has 3,317 associated author(s) and 15,900 associated document(s) as of 2022-11-01


La clase AffiliationRetrieval esta compuesta por:
* **address**
La dirección de la afiliación.

* **affiliation_name**
El nombre de la afiliación.

* **author_count**
Número de autores asociados a la afiliación.

* **city**
La ciudad de la afiliación.

* **country**
El país de la afiliación.

* **date_created**
Fecha de creación del registro de Scopus.

* **document_count**
Número de documentos de la afiliación.

* **eid**
El EID de la afiliación. Represenado en wikidata a través de la propiedad [P1154](https://www.wikidata.org/wiki/Property:P1154 )

* **identifier**
El ID de Scopus de la afiliación.

* **name_variants**
Una lista de namedtuples que representan variantes del nombre de la afiliación con el número de documentos que se refieren a esta variante.

* **org_domain**
Tipo de afiliación. Requiere la vista ESTÁNDAR y sólo está presente si el perfil es de tipo org.

* **org_type**
Sitio web de la afiliación. Requiere la vista STANDARD.

* **org_URL**
Sitio web de la afiliación. Requiere la vista STANDARD.

* **postal_code**
El código postal de la afiliación. Requiere la vista STANDARD.

* **scopus_affiliation_link**
Enlace a la vista web de Scopus de la afiliación.

* **self_link**
Enlace a la página API de la afiliación.

* **search_link**
URL to the API page listing documents of the affiliation.

* **state**
Estado (subunidad administrativa del país) de la afiliación. Requiere la vista STANDARD.

* **sort_name**
El nombre de la afiliación utilizado para la clasificación. Requiere la vista STANDARD.

* **url**
URL de la página API de la afiliación.


En la interfaz web de Scopus se puede observar que las instituciones se representan en una jerarquía y también se relacionan entre si a partir de una relación de colaboración. 

 ![Scopus Affiliation Hierarchy](assets/images/scopus_affiliation_hierarchy.jpeg)

Al parecer no se puede obtener estos datos a partir de la API por lo que se va a utilizar la [AuthorRetrieval](https://pybliometrics.readthedocs.io/en/stable/classes/AuthorRetrieval.html) para que a través de [AuthorRetrievalAPI](https://dev.elsevier.com/documentation/AuthorRetrievalAPI.wadl) obtener los autores cuyos trabajos tengan filiación UNLP. Una vez obtenido los autores, se obtendrá las filiaciones a las cuales pertenece cada uno, para agregarlas a un datamart.


In [6]:
from pybliometrics.scopus import AuthorSearch
author_search = AuthorSearch('AF-ID(60032057)')
pd.set_option('display.max_columns', None)
df = pd.DataFrame(author_search.authors)
aff_ids = df['affiliation_id'].unique()

Itero sobre los identificadores obtenidos y recupero cada una de las afiliaciones asociadas a los autores UNLP

In [12]:
for id in aff_ids:
    affilliations.append(AffiliationRetrieval(id))

Scopus401Error: The requestor is not authorized to access the requested view or fields of the resource